# Determining the Probability of College Admittance with Logistic Regression

Here, we are going to find determine the probability of a given student being admitted to college based on both their SAT Score and Gender.

### Binary Predictors in a Logistic Regression

In the same way we created dummies for a linear regression, we can use binary predictors in a logisitic regression. 

### Load the Relevant Libraries

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Load the Data

In [8]:
raw_data = pd.read_csv('2.02. Binary predictors.csv')
raw_data

,SAT,Admitted,Gender
0,1363,No,Male
1,1792,Yes,Female
2,1954,Yes,Female
3,1653,No,Male
4,1593,No,Male
...,...,...,...
163,1722,Yes,Female
164,1750,Yes,Male
165,1555,No,Male
166,1524,No,Male


Now, we have one additional piece of information - Gender. So, let's map them in a binary way with Female being 1 and Male being 0. So, male is the baseline or the reference group in statistical lingo. 

In [11]:
data = raw_data.copy()
data['Admitted'] = data['Admitted'].map({'Yes':1, 'No':0})
data['Gender'] = data['Gender'].map({'Female':1, 'Male':0})
data

,SAT,Admitted,Gender
0,1363,0,0
1,1792,1,1
2,1954,1,1
3,1653,0,0
4,1593,0,0
...,...,...,...
163,1722,1,1
164,1750,1,0
165,1555,0,0
166,1524,0,0


### Declare the Dependent and Independent Variables

In [15]:
y = data['Admitted']
x1 = data['Gender']

### Logistic Regression

In [18]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.572260
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      166
Method:                           MLE   Df Model:                            1
Date:                Tue, 13 Aug 2024   Pseudo R-squ.:                  0.1659
Time:                        18:11:29   Log-Likelihood:                -96.140
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 6.283e-10
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6436      0.222     -2.901      0.004      -1.078      -0.209
Gender         2.0786      0.363      5.727      0.000       1.367       2.790
==============================================================================
"""

The model is significant (LLR p-value < 0.05), and the 'Gender' variable (P>|z| is 0.000) is significant too. 

### Interpreting the Model

log(odds) = -0.64 + 2.08 * Gender

Using what we learned in the last lesson, we can two equations:

log(odds2) = -0.64+2.08*Gender2
log(odds1) = -0.64+2.08*Gender1

log(odds2/odds1) = 2.08*(Gender2 - Gender1)

Now, however, gender is only two possible values, 1 and 0. So, there is only a unit change, no other option. Thus, let Gender2 be equal to 1 and Gender1 be equal to 0:

log(oddsFemale/oddsMale) = 2.08*(1-0) = 2.08

In [24]:
np.exp(2.0786)

7.993270498536442

So, taking the exponential of both sides, we get the odds of a female to get admitted are 7.99 times the odds of a male:

oddsFemale = 7.99*oddsMale

This is the interpretation of Binary predictors coefficients. It is like that of continuous predictors, but actually simpler.

### New Regression with Both Predictors

Now, we do know that there is a strong relationship between SAT score and admittance, right?

So, let's create a new regression including both predictors:

In [29]:
y = data['Admitted']
x1 = data[['SAT','Gender']]

In [31]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.120117
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      165
Method:                           MLE   Df Model:                            2
Date:                Tue, 13 Aug 2024   Pseudo R-squ.:                  0.8249
Time:                        18:24:03   Log-Likelihood:                -20.180
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 5.118e-42
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -68.3489     16.454     -4.154      0.000    -100.598     -36.100
SAT            0.0406      0.010      4.129      0.000       0.021       0.060
Gender         1.9449      0.846      2.299      0.022       0.287       3.603
==============================================================================

Possibly complete quasi-separation: A fraction 0.27 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

What we get is a regression, which has a much higher Log-liklihood: -20.180(new) vs. -96.140(old), meaning it is a better model. 

This makes sense because SAT score is a very good predictor. 

We can see that the gender variable is significant P>|z|: 0.022(new), but we no longer have 0.000 from the old one. The new coefficient for Gender is 1.94, and the exponential of that is 6.99. 

In [34]:
np.exp(1.9449)

6.992932526814459

The interpretation - given the same SAT score, a female has 7 times higher odds to get admitted than a Male. 

### Understanding the Information

All this is pretty neat, but we must always think about the information that is revealed to us through these methods:

It seems that in this particular university, or this particular degree, it is much easier for females to enter.

In the real-world this is an incredibly common situation - universities places quotas on students, as they are pursuing equality. However, professions like 'Communications' are predominantly female, while STEM is predominantly male. 

Therefore, if a man applies for Communications, his odds may be 10 times higher than that of a female. And vice versa, when a femal applies to STEM, it will be much easier for her to get admitted than a male with the same SAT score. 

In this example, we don't have much context, but in real-life situations we certainly do. 

Always interpret the results considering the context. If they seem illogical, it may be worth revisiting your model, rather than blindly following it. 

### Calculating the Accuracy of the Model

In [42]:
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      165
Method:                           MLE   Df Model:                            2
Date:                Wed, 14 Aug 2024   Pseudo R-squ.:                  0.8249
Time:                        15:21:45   Log-Likelihood:                -20.180
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 5.118e-42
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -68.3489     16.454     -4.154      0.000    -100.598     -36.100
SAT            0.0406      0.010      4.129      0.000       0.021       0.060
Gender         1.9449      0.846      2.299      0.022       0.287       3.603
==============================================================================

Possibly complete quasi-separation: A fraction 0.27 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

We said that there is this Pseudo R-Squared that give us a "pseudo sense" of how well our regression is doing. 

In practice, though, there are better ideas of how to assess the accuracy of the model:

-We have a model that predicts values 

-We also have the actual values (data['Admitted']) that were observed

So, we can surely check the accuracy. We already saw the sm.LogitResults.predict() method, which returns the values predicted by our model. 

If we write:

In [45]:
results_log.predict()

array([2.24098643e-06, 9.98264069e-01, 9.99997581e-01, 2.25470272e-01,
       2.48392751e-02, 9.92249420e-01, 9.96544212e-01, 9.99963261e-01,
       9.99971204e-01, 1.48031753e-02, 9.99875812e-01, 9.99951185e-01,
       7.60867651e-01, 2.33384671e-06, 5.96283811e-01, 9.99834996e-01,
       1.14446654e-01, 1.18626448e-01, 5.05147726e-01, 9.99865308e-01,
       9.99999366e-01, 9.99997048e-01, 1.71939595e-04, 5.61635704e-03,
       9.68663798e-01, 9.99644611e-01, 4.84851641e-01, 9.91962775e-01,
       9.99828160e-01, 9.94609023e-01, 1.15028367e-04, 8.32585363e-01,
       2.47449367e-01, 9.99998840e-01, 9.98847293e-01, 9.99372736e-01,
       3.12716933e-01, 9.99932453e-01, 2.32639633e-01, 5.29744519e-05,
       1.95739604e-02, 4.54521689e-01, 9.99956956e-01, 2.97763113e-06,
       9.94178832e-01, 1.77714430e-05, 9.93914956e-01, 2.29360536e-04,
       3.30501192e-04, 6.89914934e-03, 4.24966754e-03, 9.99999657e-01,
       9.23952460e-01, 2.28569785e-02, 9.99994550e-01, 5.47478329e-06,
      

We get all of the predicted values. 

Let's apply some formatting to make them more legible:

In [48]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
results_log.predict()

array([0.00, 1.00, 1.00, 0.23, 0.02, 0.99, 1.00, 1.00, 1.00, 0.01, 1.00,
       1.00, 0.76, 0.00, 0.60, 1.00, 0.11, 0.12, 0.51, 1.00, 1.00, 1.00,
       0.00, 0.01, 0.97, 1.00, 0.48, 0.99, 1.00, 0.99, 0.00, 0.83, 0.25,
       1.00, 1.00, 1.00, 0.31, 1.00, 0.23, 0.00, 0.02, 0.45, 1.00, 0.00,
       0.99, 0.00, 0.99, 0.00, 0.00, 0.01, 0.00, 1.00, 0.92, 0.02, 1.00,
       0.00, 0.37, 0.98, 0.12, 1.00, 0.00, 0.78, 1.00, 1.00, 0.98, 0.00,
       0.00, 0.00, 1.00, 0.00, 0.78, 0.12, 0.00, 0.99, 1.00, 1.00, 0.00,
       0.30, 1.00, 1.00, 0.00, 1.00, 1.00, 0.85, 1.00, 1.00, 0.00, 1.00,
       1.00, 0.89, 0.83, 0.00, 0.98, 0.97, 0.00, 1.00, 1.00, 0.03, 0.99,
       0.96, 1.00, 0.00, 1.00, 0.01, 0.01, 1.00, 1.00, 1.00, 0.00, 0.00,
       0.02, 0.33, 0.00, 1.00, 0.09, 0.00, 0.97, 0.00, 0.75, 1.00, 1.00,
       0.01, 0.01, 0.00, 1.00, 0.00, 0.99, 0.57, 0.54, 0.87, 0.83, 0.00,
       1.00, 0.00, 0.00, 0.00, 1.00, 0.04, 0.00, 0.01, 1.00, 0.99, 0.52,
       1.00, 1.00, 0.05, 0.00, 0.00, 0.00, 0.68, 1.

So, we see there there are some 0s, 1s, and some values in between those. As we discussed earlier, those are probabilities (i.e. the values of Pi in the model, which is the probability of being admitted). 

Ultimately, values less than 0.5 means there is less than 50% chance of admission, so we would round down to 0. Alternatively, values above 0.5 would be rounded up to 1. 

Using this simplification, what we can do is compare the actual values we observed with those that the model spit out:

In [51]:
np.array(data['Admitted'])

array([0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0])

If 80% of the predicted values coincide with the actual values, we say the model has 80% accuracy. 

Now, these are a lot of values to compare, so let's summarize it in a table, using sm.LogitResults.pred_table() method, which returns a table that compares predicted and actual values:

In [54]:
results_log.pred_table()

array([[69.00, 5.00],
       [4.00, 90.00]])

What we get is this little matrix with four entries.

Let's format it better, so we can discuss it further:

In [59]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0:'Actual 0', 1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,69.0,5.0
Actual 1,4.0,90.0


The table has a respectable name, Confusion Matrix - shows how confused out model was. 

Here is how to read it:

For 69 observations, the model predicted 0 and the true value was 0. 

For 90 observations the model predicted 1 and the true value was 1. (These cells indicate in how many cases the model did it's job well)

However, in 4 observations, the model predicted 0 while the true value was 1. Similarly, in 5 observations, the model predicted 1 and the true value was 0. (These are the cases where the model 'got confused')

Now, there are many different metrics we can calculate from this matrix. 

-The most important one is the accuracy of the model: In 69 + 90 = 159 cases, the model did its job well. In 5 + 4 = 9 cases, the model was incorrect. So, overall, the model made an accurate prediction in 159 out of 168 cases, so (159/168) = .946 or 94.6% accuracy of the model. So our model seems very good at classifying!

Here is a short piece of code that will calculate the accuracy for you:

In [63]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.9464285714285714

For homework, you can calculate the accuracy of all the models we have done so far in this section.

### Testing the Model

To conslude the topic, we must test our model.

As we said before, testing is done on a dataset that the model has never seen before. So, let's load a new CSV file called '2.03. Test Dataset' in the variable test:

In [69]:
test = pd.read_csv('2.03. Test dataset.csv')
test

,SAT,Admitted,Gender
0,1323,No,Male
1,1725,Yes,Female
2,1762,Yes,Female
3,1777,Yes,Male
4,1665,No,Male
5,1556,Yes,Female
6,1731,Yes,Female
7,1809,Yes,Female
8,1930,Yes,Female
9,1708,Yes,Male


It looks in the same wasy as the raw data. Actually, they were part of the same file, but it has been split into 2. The regression was trained on the first one, so now we will test it. 

Splitting the dataset is a very common practice and an almost compulsory one. Our split was 90% - 10%, thus there are 19 observations for the test. 

Next, let's map the categorical variables like we already know how to do:

In [72]:
test['Admitted'] = test['Admitted'].map({'Yes':1, 'No':0})
test['Gender'] = test['Gender'].map({'Female':1, 'Male':0})
test

,SAT,Admitted,Gender
0,1323,0,0
1,1725,1,1
2,1762,1,1
3,1777,1,0
4,1665,0,0
5,1556,1,1
6,1731,1,1
7,1809,1,1
8,1930,1,1
9,1708,1,0


Now what we will do is:

1.) Use our model to make predictions based on the test data

2.) Compate those with the actual outcome

3.) Calculate the accuracy

4.) Create a confusion matrix on our own

In order to predict values using the Statsmodels method sm.LogitResults.predict() we used ealier, our test data should look the same as the input data in which the regression was trained. 

In [76]:
x

,const,SAT,Gender
0,1.0,1363,0
1,1.0,1792,1
2,1.0,1954,1
3,1.0,1653,0
4,1.0,1593,0
...,...,...,...
163,1.0,1722,1
164,1.0,1750,0
165,1.0,1555,0
166,1.0,1524,0


After printing the variable x, we see that there are three columns - Constant, SAT, and Gender in that order. 

Rememeber, order is very important because the coefficients of the regression will expect it. If we fail to deliver the correct order, the prediction will be wrong. 

Now, we will create a new variable called test_actual, where we will store the admission information - these are the actual outcomes that were observed. 

Then, we'll declare another variable test_data, which will containt everything from test, except the admission information, leaving us with 'SAT' and 'Gender'. Then, add a constant using the sm.add_constant() method:

In [84]:
test_actual = test['Admitted']
test_data = test.drop(['Admitted'], axis=1)
test_data = sm.add_constant(test_data)
#To reorder columns to match those of x:
#test_data = test_data[x.columns.values]
test_data

,const,SAT,Gender
0,1.0,1323,0
1,1.0,1725,1
2,1.0,1762,1
3,1.0,1777,0
4,1.0,1665,0
5,1.0,1556,1
6,1.0,1731,1
7,1.0,1809,1
8,1.0,1930,1
9,1.0,1708,0


Now, the data looks exactly like the input data. Note that we got a bit lucky here, as the variables remained in the same row order. Normally, you would have to reorder the columns to match those of x - see the # comment above as a reference if you ever need to do this.

Now, the final step is to create the confusion matrix. Unfortunately, sm.LogitResults.pred_table() does not provide testing as a functionality, unlike Sci-kit Learn. Either way, it is not very hard to do.

In the last lesson, we explained that the confusion matrix is simply a place to summarize otherwise available values. So, we will manually create a function called confusion_matrix:

In [87]:
def confusion_matrix(data,actual_values,model):

    pred_values = model.predict(data)
    bins=np.array([0,0.5,1])
    cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
    accuracy = (cm[0,0]+cm[1,1])/cm.sum()
    return cm, accuracy

There are three arguments - the input data, the actual values, and the model. 

Our function will use the already created regression model to make predictions based on the data (pred_values). 

Then, it will summarize the values in a table (cm).

Finally, we'll make it calculate the accuracy too.

You can find code with detailed comments about the function above in course resources.

Once run, the function will return the confusion matrix and the accuracy. 

Now, we will test the model - we will let cm equal to the function confusion_matrix with arguments - data, actual values, and model - equal to test_data, test_actual, and results_log, respectively:

In [92]:
cm = confusion_matrix(test_data, test_actual, results_log)
cm

(array([[5.00, 1.00],
        [1.00, 12.00]]),
 0.8947368421052632)

The array is the confusion matrix and the 0.894 number or 89.47% is the accuracy. 

89.47% is also the figure we use to refer to the overall accuracy of the regression. 

Almost always, the training accuracy (94.64%) is higher than the test accuracy (89.47%). This is because of the overfitting we talked about - the regression fitted the training data as well as possible, but that doesn't mean that the prediction is true for all the values from the population. 

This is why we test data the model has never seen and make our conclusions on that. 

Finally, we can format the confusion matrix, should you want to examine it further:

In [97]:
cm_df = pd.DataFrame(cm[0])
cm_df.columns = ['Predicted 0', 'Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0', 1: 'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,5.0,1.0
Actual 1,1.0,12.0


By the way, the opposite of accuracy, is called the misclassification rate, which equals the # of misclassified elements / # of all elements. 

In this case it is: (1+1)/19 = 0.1052 or 10.53%

In [100]:
print('Misclassification Rate: '+str((1+1)/19))

Misclassification Rate: 0.10526315789473684
